# Cifar-10 이미지 분류
### 전체 데이터 사용
### Data Augmentation 적용
### 출처: [Achieving 90% accuracy in Object Recognition ](https://appliedmachinelearning.blog/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/)

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.regularizers import l2
import numpy as np

#### 자료형 변환 및 스케일링
- X: 실수형으로 정규화
- Y: 1-hot encoding
    * airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

In [14]:
(X_train, y_train0), (X_test, y_test0) = cifar10.load_data()
print(X_train.shape, X_train.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test.shape, X_test.dtype)
print(y_test0.shape, y_test0.dtype)

(50000, 32, 32, 3) uint8
(50000, 1) uint8
(10000, 32, 32, 3) uint8
(10000, 1) uint8


In [15]:
# Full test 
# Train set [0:9999]
# test set [10000:12499]        

# 훈련셋과 검증셋 분리
X_train = X_train[0:10000]
y_train0 = y_train0[0:10000]
X_test = X_test[0:2500] # 훈련셋의 30%를 검증셋으로 사용
y_test0 = y_test0[0:2500]

In [16]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

print(X_train.shape, X_train.dtype)

(10000, 32, 32, 3) float32


In [17]:
Y_train = tf.keras.utils.to_categorical(y_train0, 10)
Y_test = tf.keras.utils.to_categorical(y_test0, 10)
Y_train[:4]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

#### 모형 구현

In [18]:
np.random.seed(0)
weight_decay = 1e-4

In [19]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay), 
                 input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0

In [20]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(lr=0.001, decay=weight_decay), 
              metrics=['accuracy'])

In [21]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    if epoch > 40:
        lrate = 0.0003
    return lrate

In [27]:
#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(X_train)

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
modelpath = "../model/cifar10-cnn-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [36]:
from tensorflow.keras.callbacks import LearningRateScheduler

history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                    steps_per_epoch=X_train.shape[0] // 64, epochs=10,
                    verbose=2, validation_data=(X_test, Y_test),
                    #callbacks=[LearningRateScheduler(lr_schedule)])
                    callbacks=[early_stopping_callback, checkpointer])                             
                             

  ...
    to  
  ['...']
Train for 156 steps, validate on 2500 samples
Epoch 1/10

Epoch 00001: val_loss improved from inf to 0.93851, saving model to ../model/cifar10-cnn-01-0.9385.hdf5
156/156 - 32s - loss: 0.8480 - accuracy: 0.7353 - val_loss: 0.9385 - val_accuracy: 0.7248
Epoch 2/10

Epoch 00002: val_loss did not improve from 0.93851
156/156 - 32s - loss: 0.8256 - accuracy: 0.7408 - val_loss: 1.0940 - val_accuracy: 0.7084
Epoch 3/10

Epoch 00003: val_loss did not improve from 0.93851
156/156 - 32s - loss: 0.8143 - accuracy: 0.7461 - val_loss: 1.0364 - val_accuracy: 0.7148
Epoch 4/10

Epoch 00004: val_loss improved from 0.93851 to 0.90182, saving model to ../model/cifar10-cnn-04-0.9018.hdf5
156/156 - 32s - loss: 0.8012 - accuracy: 0.7499 - val_loss: 0.9018 - val_accuracy: 0.7300
Epoch 5/10

Epoch 00005: val_loss did not improve from 0.90182
156/156 - 32s - loss: 0.7790 - accuracy: 0.7572 - val_loss: 0.9361 - val_accuracy: 0.7280
Epoch 6/10

Epoch 00006: val_loss did not improve from

#### 모델 평가하기

In [43]:
scores = model.evaluate(X_test, Y_test, batch_size=128, verbose=2)
print('\nAccuracy: %.4f' % scores[1])

2500/2500 - 1s - loss: 1.0739 - accuracy: 0.7160

Accuracy: 0.7160


#### 모델 저장하기

In [37]:
model.save_weights('../model/cifar10-full-v3.h5') 